# **Detection of Covid-19**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import * 
from keras.preprocessing import image

#  Model deployment

In [ ]:
#getting Data
from google.colab import drive
drive.mount("/content/drive")
!unzip '/content/drive/My Drive/Val.zip'
!unzip '/content/drive/My Drive/train'

In [ ]:
#Training model
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
#Getting parameters
model.summary()

# Training data 

In [ ]:
#Moulding train images
train_datagen = image.ImageDataGenerator(rescale = 1./255, shear_range = 0.2,zoom_range = 0.2, horizontal_flip = True)

test_dataset = image.ImageDataGenerator(rescale=1./255)

In [ ]:
#Reshaping test and validation images 
train_generator = train_datagen.flow_from_directory(
    '/content/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')
validation_generator = test_dataset.flow_from_directory(
    '/content/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

In [ ]:
#Training the model
hist_new = model.fit_generator(
    train_generator,
    steps_per_epoch=5,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps=2
)

In [ ]:
#Getting summary
summary=hist_new.history
print(summary)

In [ ]:
model.save("covid.h5")

In [ ]:
model.evaluate_generator(train_generator)

In [ ]:
print(model.evaluate_generator(validation_generator))

## Confusion Matrix

In [ ]:
import os
train_generator.class_indices

In [ ]:
y_actual, y_test = [],[]

In [ ]:
for i in os.listdir("/content/Val/Normal/"):
    img=image.load_img("/content/Val/Normal/"+i,target_size=(224,224))
    img=image.img_to_array(img)
    img=np.expand_dims(img,axis=0)
    pred=model.predict_classes(img)
    y_test.append(pred[0,0])
    y_actual.append(1)

In [ ]:
for i in os.listdir("/content/Val/Covid/"):
    img=image.load_img("/content/Val/Covid/"+i,target_size=(224,224))
    img=image.img_to_array(img)
    img=np.expand_dims(img,axis=0)
    pred=model.predict_classes(img)
    y_test.append(pred[0,0])
    y_actual.append(0)

In [ ]:
y_actual=np.array(y_actual)
y_test=np.array(y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
cn=confusion_matrix(y_actual,y_test)

In [ ]:
sns.heatmap(cn,cmap="plasma",annot=True) #0: Covid ; 1: Normal